### A simple play with the CIFAR-10 dataset and keras

In [28]:
from keras.datasets import cifar10
import keras

from keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Activation
from keras.models import Sequential
from keras.optimizers import RMSprop

In [12]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

### Quick analysis

In [13]:
print("xtrain =>", x_train.shape)
print("ytrain =>", y_train.shape)
print("xtest =>", x_test.shape)
print("ytest =>", y_test.shape)

xtrain => (50000, 32, 32, 3)
ytrain => (50000, 1)
xtest => (10000, 32, 32, 3)
ytest => (10000, 1)


In [14]:
y_train[0:10]

array([[6],
       [9],
       [9],
       [4],
       [1],
       [1],
       [2],
       [7],
       [8],
       [3]], dtype=uint8)

In [15]:
x_train[0, 1]

array([[ 16,  20,  20],
       [  0,   0,   0],
       [ 18,   8,   0],
       [ 51,  27,   8],
       [ 88,  51,  21],
       [120,  82,  43],
       [128,  89,  45],
       [127,  86,  44],
       [126,  87,  50],
       [116,  79,  44],
       [106,  70,  37],
       [101,  67,  35],
       [105,  70,  36],
       [113,  74,  35],
       [109,  70,  33],
       [112,  72,  37],
       [119,  79,  44],
       [109,  71,  33],
       [105,  69,  27],
       [125,  89,  46],
       [127,  92,  46],
       [122,  85,  39],
       [131,  89,  47],
       [124,  82,  41],
       [121,  79,  37],
       [131,  89,  48],
       [132,  91,  53],
       [133,  94,  58],
       [133,  96,  60],
       [123,  88,  55],
       [119,  83,  50],
       [122,  87,  57]], dtype=uint8)

### Time to normalize the data...

In [17]:
X_train = x_train / 255
X_test = x_test / 255

In [19]:
X_train[0, 1]

array([[0.0627451 , 0.07843137, 0.07843137],
       [0.        , 0.        , 0.        ],
       [0.07058824, 0.03137255, 0.        ],
       [0.2       , 0.10588235, 0.03137255],
       [0.34509804, 0.2       , 0.08235294],
       [0.47058824, 0.32156863, 0.16862745],
       [0.50196078, 0.34901961, 0.17647059],
       [0.49803922, 0.3372549 , 0.17254902],
       [0.49411765, 0.34117647, 0.19607843],
       [0.45490196, 0.30980392, 0.17254902],
       [0.41568627, 0.2745098 , 0.14509804],
       [0.39607843, 0.2627451 , 0.1372549 ],
       [0.41176471, 0.2745098 , 0.14117647],
       [0.44313725, 0.29019608, 0.1372549 ],
       [0.42745098, 0.2745098 , 0.12941176],
       [0.43921569, 0.28235294, 0.14509804],
       [0.46666667, 0.30980392, 0.17254902],
       [0.42745098, 0.27843137, 0.12941176],
       [0.41176471, 0.27058824, 0.10588235],
       [0.49019608, 0.34901961, 0.18039216],
       [0.49803922, 0.36078431, 0.18039216],
       [0.47843137, 0.33333333, 0.15294118],
       [0.

### Now, we need to convert the y-values into matrices of 10 columns (10 classes...)

In [22]:
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

In [23]:
print("y_train => ", y_train.shape)

y_train =>  (50000, 10)


In [27]:
y_train[0:5, ]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [35]:
model = Sequential()

# 32x32x3 to 32x32x6
model.add(Conv2D(6, 
                 kernel_size=(3,3), 
                 strides=(1, 1), 
                 padding='same', 
                 dilation_rate=(1, 1), 
                 activation='relu',
                 input_shape=(32,32,3),
                 use_bias=True))

# 32x32x6 to 16x16x6
model.add(MaxPooling2D(pool_size=(2, 2)))


# 16x16x6 to 16x16x18
model.add(Conv2D(18, 
                 kernel_size=(3,3), 
                 strides=(1, 1), 
                 padding='same', 
                 dilation_rate=(1, 1), 
                 activation='relu',
                 input_shape=(32,32,3),
                 use_bias=True))

# 16x16x18 to 8x8x18
model.add(MaxPooling2D(pool_size=(2, 2)))


# 8x8x18 to 8x8x36
model.add(Conv2D(36, 
                 kernel_size=(3,3), 
                 strides=(1, 1), 
                 padding='same', 
                 dilation_rate=(1, 1), 
                 activation='relu',
                 input_shape=(32,32,3),
                 use_bias=True))

# 8x8x36 to 4x4x36
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten 4x4x36 to 1x576 (4*4*36)
model.add(Flatten())

# An extra layer of 512
model.add(Dense(512))

# Output layer
model.add(Dense(10)) # 10 -> Number of classes
model.add(Activation('softmax'))

In [36]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
metrics=['accuracy'])

In [38]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 32, 32, 6)         168       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 16, 16, 6)         0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 16, 16, 18)        990       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 8, 8, 18)          0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 8, 36)          5868      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 4, 4, 36)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 576)               0         
__________

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=64,
                    epochs=10,
                    verbose=1,
                    validation_data=(X_test, y_test))


Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 30s 601us/step - loss: 1.8855 - acc: 0.3309 - val_loss: 1.6921 - val_acc: 0.3948